# 쿼리 라우팅 1.
- 메타데이터 기반 라우터
- 노드레벨 라우팅 기능
- 메타데이터 기반 필터로 불필요한 노이즈 감소 (성능 up, 비용 down)

In [ ]:
!pip install llama_index openai qdrant_client llama-index-vector-stores-qdrant

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}" f"**Text:** "
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown(""))

In [ ]:
# set up OpenAI
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
import openai

openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
from llama_index.core.indices.vector_store.base import VectorStoreIndex
from llama_index.vector_stores.qdrant import QdrantVectorStore

import qdrant_client
from qdrant_client import models
client = qdrant_client.QdrantClient(
    url="",
    api_key="",
)


In [ ]:
from llama_index.core import VectorStoreIndex, StorageContext

In [ ]:
from llama_index.core.schema import TextNode

nodes = [
    TextNode(
        text=(
            "A bunch of scientists bring back dinosaurs and mayhem breaks"
            " loose"
        ),
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    TextNode(
        text=(
            "Leo DiCaprio gets lost in a dream within a dream within a dream"
            " within a ..."
        ),
        metadata={
            "year": 2010,
            "director": "Christopher Nolan",
            "rating": 8.2,
        },
    ),
    TextNode(
        text=(
            "Batman now serves as a under-the-hood hero in Gotham City..."
        ),
        metadata={
            "year": 2008,
            "director": "Christopher Nolan",
            "rating": 8.9,
        },
    ),
    TextNode(
        text=(
            "A psychologist / detective gets lost in a series of dreams within"
            " dreams within dreams and Inception reused the idea"
        ),
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    TextNode(
        text=(
            "A bunch of normal-sized women are supremely wholesome and some"
            " men pine after them"
        ),
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    TextNode(
        text="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
]

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

Settings.embed_model = OpenAIEmbedding(
    model="text-embedding-3-small"
)

Settings.llm= OpenAI(temperature=0,model='gpt-4o-mini')

In [ ]:
vector_store = QdrantVectorStore(client=client, collection_name="metafilter_exercise")
storage_context = StorageContext.from_defaults(vector_store=vector_store)


In [ ]:
index = VectorStoreIndex

In [ ]:
from llama_index.core.retrievers import VectorIndexAutoRetriever
from llama_index.core.vector_stores import MetadataInfo, VectorStoreInfo

vector_store_info = VectorStoreInfo(
    content_info="Brief summary of a movie",
    metadata_info=[
        MetadataInfo(
            name="genre",
            description="The genre of the movie",
            type="string or list[string]",
        ),
        MetadataInfo(
            name="year",
            description="The year the movie was released",
            type="integer",
        ),
        MetadataInfo(
            name="director",
            description="The name of the movie director",
            type="string",
        ),
        MetadataInfo(
            name="rating",
            description="A 1-10 rating for the movie",
            type="float",
        ),
    ],
)

# 메타데이터 필터용 리트리버(오토리트리버) 설정
retriever =

In [ ]:
# 메타데이터 operand 활용하여 1개만 뜨는지 확인
retriever.retrieve(
    "What are 2 movies by Christopher Nolan were made before 2010?"
)

In [ ]:
# 두개 다 떠야할 경우 확인
retriever.retrieve(
    "What are 2 movies by Christopher Nolan were made before 2020?"
)

In [ ]:
# 한개 떠야할 경우 확인
retriever.retrieve(
    "What are the movies by Satoshi Kon before 2009?"
)

In [ ]:
# 아무것도 안떠야 하는 경우 확인
retriever.retrieve("Has Andrei Tarkovsky directed any science fiction movies")

In [ ]:
from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine

# 쿼리 엔진 붙히기
response_synthesizer = get_response_synthesizer()

#index를 direct하게 쓸때는 as_query_engine, retriever를 query engine으로 쓸때는 RetrieverQueryEngine
metadata_query_engine = RetrieverQueryEngine(
)


In [ ]:
# 답안까지 연계되는 behavior 확인
metadata_query_engine.query("What was the rating of the movie by Christopher Nolan that were made before 2010?")

In [ ]:
# 답안까지 연계되는 behavior 확인
metadata_query_engine.query("Has Andrei Tarkovsky directed any science fiction movies?")

In [ ]:
# Naive RAG과의 비교를 통해 메타데이터 필터 기능의 효용성 확인
query_engine =

In [ ]:
query_engine.query("What was the rating of the movie by Christopher Nolan that were made before 2010?")
# 메타데이터 필터 없이는 무조건 top-2로 retrieve되어 노이즈로써 컨텍스트 참고하게 됨.

In [ ]:
query_engine.query("Has Andrei Tarkovsky directed any science fiction movies?")
# 메타데이터 필터 없이는 무조건 top-2로 retrieve되어 노이즈로써 컨텍스트 참고하게 됨.

# Do It Yourself

In [ ]:
nodes = [
    TextNode(
        text="The Shawshank Redemption",
        metadata={
            "author": "Stephen King",
            "theme": "Friendship",
            "year": 1994,
        },
    ),
    TextNode(
        text="The Godfather",
        metadata={
            "director": "Francis Ford Coppola",
            "theme": "Mafia",
            "year": 1972,
        },
    ),
    TextNode(
        text="Inception",
        metadata={
            "director": "Christopher Nolan",
            "theme": "Fiction",
            "year": 2010,
        },
    ),
    TextNode(
        text="To Kill a Mockingbird",
        metadata={
            "author": "Harper Lee",
            "theme": "Fiction",
            "year": 1960,
        },
    ),
    TextNode(
        text="1984",
        metadata={
            "author": "George Orwell",
            "theme": "Totalitarianism",
            "year": 1949,
        },
    ),
    TextNode(
        text="The Great Gatsby",
        metadata={
            "author": "F. Scott Fitzgerald",
            "theme": "The American Dream",
            "year": 1925,
        },
    ),
    TextNode(
        text="Harry Potter and the Sorcerer's Stone",
        metadata={
            "author": "J.K. Rowling",
            "theme": "Fiction",
            "year": 1997,
        },
    ),
]

In [ ]:
# 백엔드 벡터스토어 정의: 쿼드란트 컬렉션
vector_store =
storage_context =

# 벡터스토어 인덱스에 스토리지컨텍스트로 연결
index2 =

In [ ]:
from llama_index.core.retrievers import VectorIndexAutoRetriever
from llama_index.core.vector_stores import MetadataInfo, VectorStoreInfo


vector_store_info = VectorStoreInfo(
    content_info="famous books and movies",
    metadata_info=[
        MetadataInfo(
            name="director",
            type="str",
            description=("Name of the director"),
        ),
        MetadataInfo(
            name="theme",
            type="str",
            description=("Theme of the book/movie"),
        ),
        MetadataInfo(
            name="year",
            type="int",
            description=("Year of the book/movie"),
        ),
    ],
)
#메타데이터 필터용 리트리버 설정
retriever =

In [ ]:
# 테스트 : 설정한 리트리버로 "Tell me about some books/movies after the year 2000"에 대한 리트리빙 해보기
nodes =

In [ ]:
for node in nodes:
    print(node.text)
    print(node.metadata)

In [ ]:
# 테스트 : 설정한 리트리버로 "Tell me about some books that are Fiction"에 대한 리트리빙 해보기
nodes =

In [ ]:
for node in nodes:
    print(node.text)
    print(node.metadata)

In [ ]:
# 테스트 : 설정한 리트리버로 "Tell me about some books that are mafia-themed"에 대한 리트리빙 해보기
nodes =

# 개선 방안
- 프롬프트에서 기대하는 자연어 -> 필터가 자연스럽게 연결 안될때는 메타필터 변환 프롬프트를 강화해 주면 됨

In [ ]:
from llama_index.core.prompts import display_prompt_dict
from llama_index.core import PromptTemplate

prompts_dict = retriever.get_prompts()
display_prompt_dict(prompts_dict)
prompts_dict["prompt"].template_vars

In [ ]:
# 프롬프트 템플릿 수정

prompt_tmpl_str = """\
Your goal is to structure the user's query to match the request schema provided below.

<< Structured Request Schema >>
When responding use a markdown code snippet with a JSON object formatted in the following schema:

{schema_str}

The query string should contain only text that is expected to match the contents of documents. Any conditions in the filter should not be mentioned in the query as well.

Make sure that filters only refer to attributes that exist in the data source.
Make sure that filters take into account the descriptions of attributes.
Make sure that filters are only used as needed. If there are no filters that should be applied return [] for the filter value.
If the user's query explicitly mentions number of documents to retrieve, set top_k to that number, otherwise do not set top_k.
Do NOT EVER infer a null value for a filter. This will break the downstream program. Instead, don't include the filter.

<< Example 1. >>
Data Source:
```json
{{
    "metadata_info": [
        {{
            "name": "author",
            "type": "str",
            "description": "Author name"
        }},
        {{
            "name": "book_title",
            "type": "str",
            "description": "Book title"
        }},
        {{
            "name": "year",
            "type": "int",
            "description": "Year Published"
        }},
        {{
            "name": "pages",
            "type": "int",
            "description": "Number of pages"
        }},
        {{
            "name": "summary",
            "type": "str",
            "description": "A short summary of the book"
        }}
    ],
    "content_info": "Classic literature"
}}
```

User Query:
What are some books by Jane Austen published after 1813 that explore the theme of marriage for social standing?

Additional Instructions:
None

Structured Request:
```json
{{"query": "Books related to theme of marriage for social standing", "filters": [{{"key": "year", "value": "1813", "operator": ">"}}, {{"key": "author", "value": "Jane Austen", "operator": "=="}}], "top_k": null}}

```

<< Example 2. >>
Data Source:
```json
{info_str}
```

User Query:
{query_str}

Additional Instructions:
{additional_instructions}

Structured Request:
"""

In [ ]:
# PromptTemplate으로써 가져온 템플릿 스트링 지정해주기
prompt_tmpl =

In [ ]:
# Additional Instructions에 강화 프롬프트 포함
# 강화 필요 내용: 1. xxx-themed 식의 쿼리가 Theme 필드의 기준 스트링 값으로 필터링 되도록 하기
add_instrs = """\
If input is given as 'something-themed', it means the theme is something, and you should filter it accordingly.\
"""
prompt_tmpl = prompt_tmpl.partial_format(additional_instructions=add_instrs)

In [ ]:
# additional_instructions 플레이스홀딩 필드를 기존 리트리버의 프롬프트에 업데이트 해주기


In [ ]:
# 잘 작동하지 않던 쿼리 이제 제대로 필터링 되는지 확인
nodes = retriever.retrieve("Tell me about some books that are mafia-themed")
for node in nodes:
    print(node.text)
    print(node.metadata)

In [ ]:
nodes

In [ ]:
# 고쳐야 할 부분 고쳐보자
# 강화 필요 내용:
# 1. xxx-themed 식의 쿼리가 Theme 필드의 기준 스트링 값으로 필터링 되도록 하기
# 2. 테마 필터링 시 기준 스트링의 첫글자 대문자로 만들기
add_instrs = """\
If input is given as 'something-themed', it means the theme is something, and you should filter it accordingly.
{이곳에 2번에 해당하는 프롬프팅을 해보세요}\
"""

prompt_tmpl = prompt_tmpl.partial_format(additional_instructions=add_instrs)
retriever.update_prompts({"prompt": prompt_tmpl})

In [ ]:
# 같은 쿼리 재시도
nodes = retriever.retrieve("Tell me about some books that are mafia-themed")
for node in nodes:
    print(node.text)
    print(node.metadata)

In [ ]:
#다른 문제 있었던 쿼리
nodes = retriever.retrieve("Tell me about some books that are Fiction")
for node in nodes:
    print(node.text)
    print(node.metadata)

In [ ]:
# 고쳐야 할 부분 고쳐보자
# 강화 필요 내용:
# 1. xxx-themed 식의 쿼리가 Theme 필드의 기준 스트링 값으로 필터링 되도록 하기
# 2. 테마 필터링 시 기준 스트링의 첫글자 대문자로 만들기
# 3. 테마라는 글자를 명시하지 않아도 뉘앙스 상 테마가 필터될수 있는 구조로 만들기
add_instrs = """\
If input is given as 'something-themed', it means the theme is something, and you should filter it accordingly.
Also, always convert the first character of the searching theme string to upper case.
{이곳에 3번에 해당하는 내용을 프롬프팅 해보세요}\
"""
prompt_tmpl = prompt_tmpl.partial_format(additional_instructions=add_instrs)
retriever.update_prompts({"prompt": prompt_tmpl})

In [ ]:
#업데이트된 프롬프트로 다시 시도
nodes = retriever.retrieve("Tell me about some books that are Fiction")
for node in nodes:
    print(node.text)
    print(node.metadata)